In [2]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import os
import cartopy
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
from cartopy.io.img_tiles import OSM
from cartopy.io.img_tiles import GoogleTiles as moa

import glob

import cartopy.feature as cfeature
import matplotlib.pyplot as plt
import xarray as xr
import pyproj
import osr
import datetime
import subprocess
import shutil
import numpy as np
from ipywidgets import interact

In [3]:
os.path.abspath('/')

'C:\\'

In [ ]:
url_base = 'https://ftp.gportal.jaxa.jp/standard/'
# url_folder = 'C:\Users\908138\githubdata\jaxa'
url_template = url_base + url_folder